In [7]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, recall_score, accuracy_score, precision_score
from sklearn.feature_selection import RFE
import shap
import catboost as cb
from tqdm import tqdm
from imblearn.over_sampling import SMOTE, SMOTENC, SMOTEN, BorderlineSMOTE, SVMSMOTE, ADASYN, KMeansSMOTE
from imblearn.under_sampling import RandomUnderSampler, NearMiss, TomekLinks, CondensedNearestNeighbour, OneSidedSelection, InstanceHardnessThreshold, AllKNN, RepeatedEditedNearestNeighbours, EditedNearestNeighbours
from imblearn.combine import SMOTEENN


ROOT_DIR = "data"
RANDOM_STATE = 110

train_data = pd.read_csv(os.path.join(ROOT_DIR, "train_mod.csv"))

In [8]:
def cat2num(X):
    non_numeric_columns = X.select_dtypes(include=['object']).columns
    # print("Non-numeric columns:", non_numeric_columns)

    encoded_columns = {}

    for column in non_numeric_columns:
        encoder = LabelEncoder()
        encoded_columns[column] = encoder.fit_transform(X[column])

    encoded_df = pd.DataFrame(encoded_columns, index=X.index)

    X = X.drop(columns=non_numeric_columns)
    X = pd.concat([X, encoded_df], axis=1)

    return X

def featuregen(train_data):
    axis = ['X', 'Y', 'Z']
    process = ['Dam', 'Fill1', 'Fill2']

    for ax in axis:
        for proc in process:
            stage1_col = f'HEAD NORMAL COORDINATE {ax} AXIS(Stage1) Collect Result_{proc}'
            stage2_col = f'HEAD NORMAL COORDINATE {ax} AXIS(Stage2) Collect Result_{proc}'
            stage3_col = f'HEAD NORMAL COORDINATE {ax} AXIS(Stage3) Collect Result_{proc}'
            new_col_1_3 = f'Head_DIFF_{ax}_Stage1&3_{proc}'
            new_col_max_min = f'Head_MinMax_{ax}_{proc}'
            
        
            train_data[new_col_1_3] = (train_data[stage1_col] - train_data[stage3_col]).abs()
            train_data[new_col_max_min] = train_data[[stage1_col, stage2_col, stage3_col]].max(axis=1) - train_data[[stage1_col, stage2_col, stage3_col]].min(axis=1)

    return train_data
                                                                                                            
def generate_stage_averages(df):
    stages = ['Stage1', 'Stage2', 'Stage3']
    
    for stage in stages:
        stage_columns = [col for col in df.columns if stage in col and ('Circle' in col or 'Line' in col)]
        df[f'{stage}_Average'] = df[stage_columns].mean(axis=1)
    
    return df
                                                                                                            
def generating_features(df):
 
    # Thickness difference
    df['Thickness_Diff_1_2'] = df['THICKNESS 1 Collect Result_Dam'] - df['THICKNESS 2 Collect Result_Dam']
    df['Thickness_Diff_2_3'] = df['THICKNESS 2 Collect Result_Dam'] - df['THICKNESS 3 Collect Result_Dam']
    df['Thickness_Std'] = df[['THICKNESS 1 Collect Result_Dam', 'THICKNESS 2 Collect Result_Dam', 'THICKNESS 3 Collect Result_Dam']].std(axis=1)
    df['Thickness_Max_Min_Diff'] = df[['THICKNESS 1 Collect Result_Dam', 'THICKNESS 2 Collect Result_Dam', 'THICKNESS 3 Collect Result_Dam']].max(axis=1) - df[['THICKNESS 1 Collect Result_Dam', 'THICKNESS 2 Collect Result_Dam', 'THICKNESS 3 Collect Result_Dam']].min(axis=1)
    df['Temperature_Change_Rate'] = df['Chamber Temp. Collect Result_AutoClave'] * df['Chamber Temp. Unit Time_AutoClave']

    return df

def generate_volume_to_speed_ratio(df):
    df['Volume_to_Speed_Ratio_Stage1'] = df['Dispense Volume(Stage1) Collect Result_Fill1'] / df['DISCHARGED SPEED OF RESIN Collect Result_Fill1']
    df['Volume_to_Speed_Ratio_Stage2'] = df['Dispense Volume(Stage2) Collect Result_Fill1'] / df['DISCHARGED SPEED OF RESIN Collect Result_Fill1']
    df['Volume_to_Speed_Ratio_Stage3'] = df['Dispense Volume(Stage3) Collect Result_Fill1'] / df['DISCHARGED SPEED OF RESIN Collect Result_Fill1']
    df['Volume_Sum_Fill1'] = df['Dispense Volume(Stage1) Collect Result_Fill1'] + df['Dispense Volume(Stage2) Collect Result_Fill1'] + df['Dispense Volume(Stage3) Collect Result_Fill1']
    
    return df

def generate_pressure_change_rate(df):
    df['Pressure_Change_Rate_1st'] = df['1st Pressure Collect Result_AutoClave'] * df['1st Pressure 1st Pressure Unit Time_AutoClave']
    df['Pressure_Change_Rate_2nd'] = df['2nd Pressure Collect Result_AutoClave'] * df['2nd Pressure Unit Time_AutoClave']
    df['Pressure_Change_Rate_3rd'] = df['3rd Pressure Collect Result_AutoClave'] * df['3rd Pressure Unit Time_AutoClave']
    
    return df

def generate_volume_to_time_ratio(df):
    df['Volume_to_Time_Ratio_Stage1'] = df['Dispense Volume(Stage1) Collect Result_Dam'] / df['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam']
    df['Volume_to_Time_Ratio_Stage2'] = df['Dispense Volume(Stage2) Collect Result_Dam'] / df['DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam']
    df['Volume_to_Time_Ratio_Stage3'] = df['Dispense Volume(Stage3) Collect Result_Dam'] / df['DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam']
    
    return df


train_data = cat2num(train_data)
train_data = featuregen(train_data)
train_data = generating_features(train_data)
train_data = generate_volume_to_speed_ratio(train_data)
train_data = generate_pressure_change_rate(train_data)
train_data = generate_volume_to_time_ratio(train_data)
train_data = generate_stage_averages(train_data)


train_data = train_data[[
    
                         'Head_DIFF_X_Stage1&3_Dam',
                         'Head_DIFF_X_Stage1&3_Fill1',
                         'Head_DIFF_X_Stage1&3_Fill2',
                         
                         'Head_MinMax_Y_Dam',
                         'Head_MinMax_Y_Fill1',
                         'Head_MinMax_Y_Fill2',
                         
                         'Stage1_Average',
                         'Stage2_Average',
                         'Stage3_Average',

                         'Thickness_Max_Min_Diff',
                         
                         'Temperature_Change_Rate',
                         
                         'Pressure_Change_Rate_1st',
                         'Pressure_Change_Rate_2nd',
                         'Pressure_Change_Rate_3rd',
                         
                         'Volume_to_Time_Ratio_Stage1',
                         'Volume_to_Time_Ratio_Stage2',
                         'Volume_to_Time_Ratio_Stage3',
    
                         'Volume_Sum_Fill1',
                         'DISCHARGED SPEED OF RESIN Collect Result_Fill1',
    
                         'target']]

In [9]:
train_data.columns

Index(['Head_DIFF_X_Stage1&3_Dam', 'Head_DIFF_X_Stage1&3_Fill1',
       'Head_DIFF_X_Stage1&3_Fill2', 'Head_MinMax_Y_Dam',
       'Head_MinMax_Y_Fill1', 'Head_MinMax_Y_Fill2', 'Stage1_Average',
       'Stage2_Average', 'Stage3_Average', 'Thickness_Max_Min_Diff',
       'Temperature_Change_Rate', 'Pressure_Change_Rate_1st',
       'Pressure_Change_Rate_2nd', 'Pressure_Change_Rate_3rd',
       'Volume_to_Time_Ratio_Stage1', 'Volume_to_Time_Ratio_Stage2',
       'Volume_to_Time_Ratio_Stage3', 'Volume_Sum_Fill1',
       'DISCHARGED SPEED OF RESIN Collect Result_Fill1', 'target'],
      dtype='object')

In [10]:
X = train_data.drop(columns=['target'])
y = train_data['target']

# 오버샘플링
oversampler = BorderlineSMOTE(sampling_strategy={0 : 10000}, random_state=RANDOM_STATE)
# borderline_smote = BorderlineSMOTE(sampling_strategy="auto", random_state=RANDOM_STATE)
# oversampler = SMOTEENN(sampling_strategy= {0 : 25000}, random_state=RANDOM_STATE)
X, y = oversampler.fit_resample(X, y)

# 언더샘플링
# undersampler = RandomUnderSampler(sampling_strategy={1: 10000}, random_state=RANDOM_STATE)
# undersampler = RandomUnderSampler(sampling_strategy="auto", random_state=RANDOM_STATE)
# undersampler = NearMiss(sampling_strategy="auto")

# undersampler = CondensedNearestNeighbour(sampling_strategy='auto')

#------------------------------------------------------------------------------------------------
# undersampler = TomekLinks(sampling_strategy='auto')
# undersampler = OneSidedSelection(sampling_strategy='auto', random_state=RANDOM_STATE)
# undersampler = AllKNN(sampling_strategy='auto')
# undersampler = RepeatedEditedNearestNeighbours(sampling_strategy='auto')
# undersampler = EditedNearestNeighbours(sampling_strategy='auto')
#------------------------------------------------------------------------------------------------
# X, y = undersampler.fit_resample(X, y)

# undersampler = NearMiss(sampling_strategy="auto")
# clf = RandomForestClassifier(random_state=RANDOM_STATE)
# undersampler = InstanceHardnessThreshold(estimator=clf, sampling_strategy='auto', random_state=RANDOM_STATE)
undersampler = RandomUnderSampler(sampling_strategy="auto", random_state=RANDOM_STATE)
X, y = undersampler.fit_resample(X, y)

# 데이터프레임으로 다시 결합
train_data = pd.concat([pd.DataFrame(X, columns=X.columns), pd.Series(y, name='target')], axis=1)

df_normal = train_data[train_data["target"] == 1]
df_abnormal = train_data[train_data["target"] == 0]
num_normal = len(df_normal)
num_abnormal = len(df_abnormal)
print(f"Total: Normal: {num_normal}, AbNormal: {num_abnormal}")

# 스케일링 (정규화 또는 표준화)
scaler = MinMaxScaler()
columns_to_scale = [col for col in train_data.columns if col != 'target']
train_data[columns_to_scale] = scaler.fit_transform(train_data[columns_to_scale])

# 데이터를 다시 피처와 타겟으로 분리
train_x = train_data.drop(columns=['target'])
train_y = train_data['target']

# 학습 및 테스트 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(
    train_x,
    train_y,
    test_size=0.22,
    random_state=RANDOM_STATE,
)

Total: Normal: 10000, AbNormal: 10000


In [11]:
train_x

,Head_DIFF_X_Stage1&3_Dam,Head_DIFF_X_Stage1&3_Fill1,Head_DIFF_X_Stage1&3_Fill2,Head_MinMax_Y_Dam,Head_MinMax_Y_Fill1,Head_MinMax_Y_Fill2,Stage1_Average,Stage2_Average,Stage3_Average,Thickness_Max_Min_Diff,Temperature_Change_Rate,Pressure_Change_Rate_1st,Pressure_Change_Rate_2nd,Pressure_Change_Rate_3rd,Volume_to_Time_Ratio_Stage1,Volume_to_Time_Ratio_Stage2,Volume_to_Time_Ratio_Stage3,Volume_Sum_Fill1,DISCHARGED SPEED OF RESIN Collect Result_Fill1
0,0.461538,0.999120,1.000000,0.008000,0.085714,1.0,0.400,0.18750,0.40,1.00000,0.580060,0.762914,0.005169,0.477774,0.015817,0.017241,0.021001,0.874687,0.972477
1,0.307692,0.998094,0.003431,0.005333,0.100000,0.0,0.200,0.62500,0.20,0.00000,0.829594,0.793229,0.340175,0.395928,0.022712,0.023319,0.022712,0.907581,0.972477
2,0.461538,0.999120,1.000000,0.008000,0.071429,1.0,0.355,0.16250,0.36,0.09292,0.563253,0.767296,0.004359,0.393435,0.987083,0.991491,0.990212,0.996867,0.972477
3,0.461538,0.999120,1.000000,0.004444,0.085714,1.0,0.500,0.28125,0.50,0.00000,0.541034,0.757862,0.004377,0.397533,0.015295,0.022006,0.015295,0.874687,0.972477
4,0.769231,0.999120,1.000000,0.004444,0.085714,1.0,0.000,0.00000,0.00,0.00000,0.794976,0.770493,0.329410,0.396731,0.021563,0.033536,0.021563,0.916353,0.972477
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0.000000,0.997654,1.000000,0.004444,0.042857,1.0,1.000,0.62500,1.00,0.00000,0.789278,0.756003,0.533827,0.395840,0.017613,0.023319,0.000000,0.947807,0.985933
19996,0.769231,0.999120,1.000000,0.000000,0.042857,1.0,0.200,0.62500,0.20,0.00000,0.725738,0.780096,0.335064,0.398777,0.022712,0.023319,0.022712,0.910401,0.972477
19997,0.384615,0.999120,1.000000,0.004444,0.042857,1.0,1.000,0.62500,1.00,0.00000,0.789278,0.786603,0.334161,0.395563,0.005732,0.023319,0.000000,0.912303,0.972477
19998,0.615385,0.998680,0.003431,0.996444,0.142857,0.0,0.500,0.18750,0.50,0.00000,0.567051,0.771690,0.004364,0.398390,0.015295,0.031051,0.015295,0.873014,0.972477


In [12]:
# Recursive Feature Elimination (RFE) for backward feature selection
model = cb.CatBoostClassifier(
    depth=4,
    iterations=400,
    l2_leaf_reg=5,
    learning_rate=0.1,
    colsample_bylevel = 0.9,
    verbose=0  # No output during training
)

# Train final model
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)

f1 = f1_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)

print(f"F1 Score: {f1:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")

#SHAP
# SHAP value caculation
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)

# Features importance
df_shap = pd.DataFrame(shap_values, columns=X_test.columns)
shap_importance = df_shap.abs().mean().sort_values(ascending=False)
print(shap_importance)

F1 Score: 0.8721
Recall: 0.9830
Accuracy: 0.8575
Precision: 0.7837
Pressure_Change_Rate_3rd                          0.893825
Volume_Sum_Fill1                                  0.470098
Volume_to_Time_Ratio_Stage1                       0.247164
Volume_to_Time_Ratio_Stage2                       0.236598
Pressure_Change_Rate_2nd                          0.222202
Stage2_Average                                    0.211941
Volume_to_Time_Ratio_Stage3                       0.201739
Pressure_Change_Rate_1st                          0.194200
Head_MinMax_Y_Fill1                               0.138213
Head_DIFF_X_Stage1&3_Dam                          0.105537
Stage3_Average                                    0.101987
Temperature_Change_Rate                           0.096608
Head_MinMax_Y_Dam                                 0.079439
Head_DIFF_X_Stage1&3_Fill2                        0.049363
Head_MinMax_Y_Fill2                               0.046295
Thickness_Max_Min_Diff                          

In [8]:
# Process test data
test_data = pd.read_csv(os.path.join(ROOT_DIR, "test_mod.csv"))
test_data = cat2num(test_data)
test_data = featuregen(test_data)
test_data = generating_features(test_data) 
test_data = generate_volume_to_speed_ratio(test_data)
test_data = generate_pressure_change_rate(test_data)
test_data = generate_volume_to_time_ratio(test_data)
test_data = generate_stage_averages(test_data)


# Scale the test data
test_data[columns_to_scale] = scaler.transform(test_data[columns_to_scale])

# Select the same features as the training data
test_x_rfe = test_data[X_train.columns]

# Predict on test data
y_pred = model.predict(test_x_rfe)

# Prepare submission
df_sub = pd.read_csv("submission.csv")
df_sub["target"] = y_pred
df_sub['target'] = df_sub['target'].map({1: 'Normal', 0: 'AbNormal'})

# Calculate the ratio of abnormal cases
counts = df_sub['target'].value_counts()
ratio = counts['AbNormal'] / (counts['AbNormal'] + counts['Normal'])
print("The ratio of abnormal is:", ratio)

# Save the submission file
df_sub.to_csv("submission.csv", index=False)

The ratio of abnormal is: 0.038707447727665455
